## Script responsável por realizar a análise descritiva

- Este código pressupõe que os arquivos do DOU estejam descompactados e se encontrem no arquivo **./saida/01_DOUSecao02_completo.parquet**
- Caso não exista o arquivo, ele pode ser gerado por meio do notebook **01_DOU_LerZips.ipynb**
- Processamento deste script:
  - Faz a leitura do arquivo **./saida/01_DOUSecao02_completo.parquet**
  - Filtra apenas portarias
  - Filtra apenas portarias do MGI
  - Cria a coluna Texto a partir da TextHTML, removento das tags html do corpo da portaria
  - Cria a coluna Total_palavras, que exibe o total de palavras da coluna Texto
  - Cria a coluna Ano com o ano da publicação (pubDate)
  - Cria a coluna Mês com o mês da publicação (pubDate)
  - Salva a saída parcial no arquivo **.\saida\02_DouSecao0<No Seção>_portarias_mgi_tratado.parquet** 
 
---


## Biblioteca

In [1]:
import pandas as pd
import string
import spacy
from bs4 import BeautifulSoup
import string
import time

In [2]:
# Função para contar palavras desconsiderando a pontuação
def contar_palavras(texto):
    # Remover pontuação
    texto_sem_pontuacao = texto.translate(str.maketrans('', '', string.punctuation))
    # Dividir o texto em palavras
    palavras = texto_sem_pontuacao.split()
    # Retornar a contagem de palavras
    return len(palavras)

In [3]:
# Função para extrair o texto de um texto HTML
def extract_text_from_html(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup.get_text(separator=' ')

In [9]:
douItem = 2
secao = f'Secao0{douItem}'

# Lê o arquivo Parquet
df = pd.read_parquet(f'./saida/01_DOU{secao}_completo.parquet', engine='pyarrow')  # ou engine='fastparquet'

# Exibe as primeiras linhas do DataFrame
df.shape


(497325, 27)

In [10]:
# Identificar as linhas com pubDate nulo
linhas_nulas = df[df['pubDate'].isna()]

# Exibir as linhas identificadas
print(linhas_nulas)

# Verificar a quantidade de linhas nulas
print(f"Número de linhas com pubDate nulo: {len(linhas_nulas)}")

Empty DataFrame
Columns: [id, name, idOficio, pubName, artType, pubDate, artClass, artCategory, artSize, artNotes, numberPage, pdfPage, editionNumber, highlightType, highlightPriority, highlight, highlightimage, highlightimagename, idMateria, Identifica, Data, Ementa, Titulo, SubTitulo, TextoHTML, file_name, zip_name]
Index: []

[0 rows x 27 columns]
Número de linhas com pubDate nulo: 0


In [11]:
df_portaria = df[df['artType'] == 'Portaria']

df_portaria.shape

(425710, 27)

In [12]:
# Filtrar os valores que contenham "ministério da inovação" na coluna 'artCategory', ignorando case
filtro = df_portaria['artCategory'].str.contains("Ministério da Gestão e da Inovação em Serviços Público", case=False, na=False)
df_portaria_mgi = df_portaria[filtro]

# Exibir o DataFrame filtrado
df_portaria_mgi.shape

(20199, 27)

In [14]:
# Início do cronômetro
start_time = time.time()

# Garantir que estamos trabalhando com uma cópia explícita
df_portaria_mgi = df_portaria_mgi.copy()

# Cria uma coluna 'Texto' com o texto sem as tagas de HTML
df_portaria_mgi.loc[:, 'Texto'] = df_portaria_mgi['TextoHTML'].apply(extract_text_from_html)

# Cria uma coluna 'Total_palavras' com o total de palavras da coluna Texto
df_portaria_mgi.loc[:, 'Total_palavras'] = df_portaria_mgi['Texto'].apply(contar_palavras)

# Converte a coluna pubDate para datetime
df_portaria_mgi['pubDate'] = pd.to_datetime(df_portaria_mgi['pubDate'], errors='coerce')

# Extrair ano e mês
df_portaria_mgi.loc[:, 'Ano'] = df_portaria_mgi['pubDate'].dt.year
df_portaria_mgi.loc[:, 'Mês'] = df_portaria_mgi['pubDate'].dt.month

# Exibe o shape do DataFrame final
print(df_portaria_mgi.shape)

# Fim do cronômetro
end_time = time.time()

# Exibir o tempo de execução
execution_time = end_time - start_time
print(f"Tempo de execução: {execution_time:.2f} segundos")

(20199, 31)
Tempo de execução: 13.47 segundos


In [15]:
# Salva saída parcial o arquivo completo com novas colunas
df.to_parquet(f'./saida/02_DOU{secao}_portarias_mgi_tratado.parquet', engine='pyarrow', index=False)

In [16]:
df_portaria_mgi.head()

,id,name,idOficio,pubName,artType,pubDate,artClass,artCategory,artSize,artNotes,...,Ementa,Titulo,SubTitulo,TextoHTML,file_name,zip_name,Texto,Total_palavras,Ano,Mês
168487,30266027,PORTARIA DE PESSOAL 529,9368436,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,None,None,"<p class=""identifica"">PORTARIA DE PESSOAL SGP/...",S02012023/529_20230125_20280233.xml.xml,S02012023.zip,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 529, DE 17...",358,2023,1
168506,30264410,ATO PORTARIA 655,9368530,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,None,None,"<p class=""identifica"">PORTARIA SGP/SEDGG/ME Nº...",S02012023/529_20230125_20280363.xml.xml,S02012023.zip,"PORTARIA SGP/SEDGG/ME Nº 655, DE 23 DE JANEIRO...",245,2023,1
168509,30265009,ATO PORTARIA DE PESSOAL 461,9368536,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,None,None,"<p class=""identifica"">PORTARIA DE PESSOAL SGP/...",S02012023/529_20230125_20280370.xml.xml,S02012023.zip,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 461, DE 16...",251,2023,1
168510,30265678,ATO PORTARIA DE PESSOAL SGP 606,9368541,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,None,None,"<p class=""identifica"">PORTARIA DE PESSOAL SGP/...",S02012023/529_20230125_20280375.xml.xml,S02012023.zip,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 606, DE 19...",251,2023,1
168512,30266030,ATO PORTARIA DE PESSOAL SGP 517,9368573,DO2,Portaria,2023-01-25,00028:00006:00000:00000:00000:00000:00000:0000...,Ministério da Gestão e da Inovação em Serviços...,12,,...,None,None,None,"<p class=""identifica"">PORTARIA DE PESSOAL SGP/...",S02012023/529_20230125_20280390.xml.xml,S02012023.zip,"PORTARIA DE PESSOAL SGP/SEDGG/ME Nº 517, DE 17...",243,2023,1
